# Imports

In [ ]:
import networkx as nx
import random
from typing import List
import pandas as pd

# Solution 1

In [ ]:
class Alu:
    def __init__(self) -> None:
        self.reset()
        self.monad = self._load_monad()

    @staticmethod
    def _load_monad() -> List[str]:
        with open("input.txt", "r") as fb:
            input_data = fb.read()
        return input_data.splitlines()

    def reset(self) -> None:
        self.w = 0
        self.x = 0
        self.y = 0
        self.z = 0
        self.phase = 0
        self.history = []

    def show_monad_logic_graph(self) -> None:
        G = nx.DiGraph()

        refs = {
            "w": None,
            "x": None,
            "y": None,
            "z": None
        }

        for n, line in enumerate(self.monad):
            # Get instruction
            instruction = line.split()

            # Split instruction
            try:
                operation, attr, value = instruction
            except:
                # operation = input
                operation, attr = instruction
                value = ""

            # Construct reference
            ref = f"{n} {operation} {attr} {value}".strip()
            
            # Get reference to previous variable reassignment of self
            self_ref = refs[attr]
            if self_ref:
                G.add_edge(self_ref, ref)
            
            # Get reference to previous variable reassignment of other variable
            if value and not value.lstrip("-").isdigit():
                other_ref = refs[value]
                if other_ref:
                    G.add_edge(other_ref, ref)

            refs[attr] = ref

        nx.spring_layout(G)
        nx.draw(G, with_labels=True)
    
    def run_monad(self, model_number: int) -> bool:
        self.reset()

        self.model_number = [int(i) for i in str(model_number)]
        if 0 in self.model_number or len(self.model_number) != 14:
            raise ValueError
        
        for line in self.monad:
            instruction = line.split()
            self._process_instruction(*instruction)

        return True if self.z == 0 else False

    def _process_instruction(self, operation, attr, value=None):
        a = getattr(self, attr)
        if value:
            try:
                b = int(value)
            except ValueError:
                b = getattr(self, value)
        else:
            b = None

        if operation == "inp":
            new_val = int(self.model_number.pop(0))
            self.phase += 1
        else:
            new_val = getattr(self, operation)(a, b)

        setattr(self, attr, new_val)
        
        self.history.append({
            "phase": self.phase,
            "instruction": (operation, attr, value),
            "a": a,
            "b": b,
            "start": a,
            "end": new_val
        })

    @staticmethod
    def add(a: int, b: int) -> int:
        return a + b

    @staticmethod
    def mul(a: int, b: int) -> int:
        return a * b

    @staticmethod
    def div(a: int, b: int) -> int:
        return a // b

    @staticmethod
    def mod(a: int, b: int) -> int:
        return a % b

    @staticmethod
    def eql(a: int, b: int) -> int:
        return 1 if a == b else 0

In [ ]:
def run_random_simulation(alu, start_range=11111111111111, end_range=99999999999999, k=1e6, interval=1e5):
    results = {}
    cache = set()
    count = 0
    while count < k:
        count += 1
        if count % interval == 0 and results:
            print(f"Count: {count}")
            lowest_z = min(results.keys())
            print(f"Lowest z: {lowest_z}")
            print(f"Based on model numbers: {results[lowest_z]}")
        val = random.choice(range(start_range, end_range))
        if val in cache:
            continue
        if "0" not in str(val):
            valid = alu.run_monad(val)
            results.setdefault(alu.z, set()).add(val)
            cache.add(val)
            if valid:
                print(val)
                print("VALID")
                break

In [ ]:
alu = Alu()

In [ ]:
run_random_simulation(alu, start_range=11111111111111, end_range=22222222222222)

In [ ]:
# (12137644217562, 350)
# (12516993815781, 350)
# (12998933147125, 350)
# (13748646417232, 351)
# (15532993921891, 353)
# (16226693179454, 354)
# (22226984246274, 376)
# (22548595352561, 376)
# (26237775326788, 381)
# (28126639715174, 382)
# (67915882321126, 483)
# (72848593585344, 506)
# (86863793296564, 536)